# MLP

Luke Ehrisman, Ben Williams

a. Load and preprocess the dataset
b. Train and test MLP model on dataset while tunning parameters to develop
multiple candidates
c. Evaluate models using appropriate metrics
d. Select most promising model
e. Predict response variable for validation inputs by using most promising moddel

In [10]:
import pandas as pd

a. Load and preprocess the dataset

In [11]:
df = pd.read_csv("data/project_adult.csv")
valid_df = pd.read_csv("data/project_validation_inputs.csv")

In [12]:
df.head()

,Unnamed: 0,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,5514,33,Local-gov,198183,Bachelors,13,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,50,United-States,>50K
1,19777,36,Private,86459,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,50,United-States,>50K
2,10781,58,Self-emp-not-inc,203039,9th,5,Separated,Craft-repair,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,32240,21,Private,180190,Assoc-voc,11,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,46,United-States,<=50K
4,9876,27,Private,279872,Some-college,10,Divorced,Other-service,Not-in-family,White,Male,0,0,40,United-States,<=50K


b. Train and test MLP model on dataset while tunning parameters to develop
multiple candidates

c. Evaluate models using appropriate metrics

d. Select most promising model

e. Predict response variable for validation inputs by using most promising model

# Reflection and Conceptual Questions


a. Why did you choose the specific architecture (e.g., number of layers, activation
functions) for each model?
b. How did you monitor and mitigate overfitting in your models?
c. What ethical concerns might arise from deploying models trained on these
datasets?
d. Why are activation functions necessary in NNs?